In [146]:
# Producxt recommender system based on user's reviews


%matplotlib inline
# Import pandas library
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# Load Amazon's data
data = pd.read_csv('1429_1.csv', low_memory=False)
data.head(3)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/martokenarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/martokenarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ


In [147]:
# get all reviews
reviews = data[['reviews.rating', 'reviews.text']]
reviews['sentiment'] = reviews['reviews.rating'].apply(lambda x :1 if x >=3 else 0)
reviews.head(3)

/Users/martokenarov/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reviews.rating,reviews.text,sentiment
0,5.0,This product so far has not disappointed. My c...,1
1,5.0,great for beginner or experienced person. Boug...,1
2,5.0,Inexpensive tablet for him to use and learn on...,1


In [159]:
reviews['reviews.text'].replace('', np.nan, inplace=True)
reviews.dropna(subset=['reviews.text'], inplace=True)

reviews_labels = reviews['sentiment'].values
reviews_text = reviews['reviews.text'].values

reviews_labels
reviews_text

/Users/martokenarov/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


array(['This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.',
       'great for beginner or experienced person. Bought as a gift and she loves it',
       'Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...',
       ...,
       "Love my Kindle Fire but I am really disappointed in the Kindle Power Fast Charging Unit. I've had it two months and I've used it many times - The first two times it worked okay but failerd on the third and many subsequent tries. I've disposed of it and use my wife's iPad Nano charger which always works just fine.",
       "I was surprised to find it did not come with any type of charging cords so I had to purchase one and then found my Sprint HTC 3D charger is faster. I would not purchase again- 1st item I've ever not liked I've purchased from Amazon",
       "to spite the fact that i have n

In [150]:
def tokenize(text): 
    return nltk.word_tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(stopwords.words("english")) 

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = False,
    ngram_range=(1, 1),
    stop_words = en_stopwords)


In [151]:
features = vectorizer.fit_transform(
    reviews_text
)
features_nd = features.toarray()

In [160]:
X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        reviews_labels,
        train_size=0.70, 
        random_state=1234)

In [161]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

In [162]:
log_model = log_model.fit(X=X_train, y=y_train)

In [163]:
y_pred = log_model.predict(X_test)

In [164]:
# import random
# j = random.randint(0,len(X_test)-7)
# for i in range(j,j+7):
#     print(y_pred[0])
#     ind = features_nd.tolist().index(X_test[i].tolist())
#     print(data[ind].strip())

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.975572225427967
